# Organization of Neural Population Code in Mouse Visual System

### Anonymous for peer-review

Table of Contents:
* **Imports**
* **Organization**
* **Stimulus Decoding**
* **Direction Decoding**
* **Decoding Results**
* **Orientation and Direction Selectivity Index Results**


_When running this notebook, note that some cells (when run for the first time) can take several minutes to hours to finish running. The notebook is configured so that after running each cell once, all progress is saved and will not require you to run it again._ 

In [ ]:
import allensdk
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
from allensdk.brain_observatory import drifting_gratings as dg

import numpy as np
import h5py
import os
import copy

import scipy
from scipy import stats
from scipy.optimize import curve_fit

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import svm
from sklearn import linear_model

import matplotlib.pyplot as plt
from matplotlib import gridspec

import statsmodels
from statsmodels.stats.multicomp import pairwise_tukeyhsd

#### Create directories to store data
Data and results are split between the stimulus category decoding and the direction decoding. 

In [ ]:
decoding_groups = ['./dvsc/stimulus_decoding/','./dvsc/direction_decoding/']
classifiers = ['svm_results/','mlr_results/']
results = ['accuracy_scores/','confusion_matrices/','interpolated_scores/','fitted_lines/']
directories = ['./dvsc/ids/','./dvsc/selectivity/osi/','./dvsc/selectivity/dsi/','./dvsc/selectivity/means/','./dvsc/figures/','./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/']

for group in decoding_groups:
    directories.append(group+'sample_size/')
    directories.append(group+'dff_matrices/')
    for classifier in classifiers:
        for result in results:
            directories.append(group+classifier+result)

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

#### Get experiment container IDs
Each experiment container contains data for one mouse across three experiments (Session A, B, C/C2).

Discard containers with:
- fewer than ten neurons recorded across three sessions (12)
- NaN and infinity values in the df/f traces (1)

In [ ]:
if os.path.exists('./dvsc/ids/all_ecs_ids.npy'):
    all_ecs_ids = list(np.load('./dvsc/ids/all_ecs_ids.npy'))
else:
    ecs = boc.get_experiment_containers()
    ecs_ids = []
    contains_nan_or_inf = []
    for container in ecs:
        exps = boc.get_ophys_experiments(experiment_container_ids=[container['id']])
        cell_list = np.intersect1d(boc.get_ophys_experiment_data(exps[0]['id']).get_cell_specimen_ids(),np.intersect1d(boc.get_ophys_experiment_data(exps[1]['id']).get_cell_specimen_ids(),boc.get_ophys_experiment_data(exps[2]['id']).get_cell_specimen_ids()))
        if len(cell_list) >= 10:
            ecs_ids.append(container['id'])  
            for cell in cell_list:
                dff_traces = np.concatenate((boc.get_ophys_experiment_data(exps[0]['id']).get_dff_traces([cell])[1][0],boc.get_ophys_experiment_data(exps[1]['id']).get_dff_traces([cell])[1][0],boc.get_ophys_experiment_data(exps[2]['id']).get_dff_traces([cell])[1][0]),axis=0)
                if np.isnan(dff_traces).any() == True or np.isinf(dff_traces).any() == True:
                    contains_nan_or_inf.append(container['id'])
    all_ecs_ids = np.delete(ecs_ids, ecs_ids.index(contains_nan_or_inf[0]))
    np.save('./dvsc/ids/all_ecs_ids', all_ecs_ids)

Stimulus Category Decoding
---
### Create the neural feature vectors
For each mouse, generate a matrix of mean dF/F values for each neuron over 10 second intervals labelled with the corresponding stimulus class.

#### Functions 
_For separating the stimulus epochs and separating them into samples of mean dF/F over ten seconds._

In [ ]:
all_stimuli = boc.get_all_stimuli()

def segment(time_data):
    pairs = []
    start = [time_data[0]]
    finish = []
    delta = np.diff(time_data)
    for i in range(len(delta)):
        if delta[i] > 100:
            finish.append(time_data[i])
            start.append(time_data[i+1])
    finish.append(time_data[-1])
    for i in range(len(start)):
        pairs.append((start[i],finish[i]))
    return (pairs)

def segment_spontaneous(time_data):
    pairs = []
    start, finish = [], []
    for i in range(len(time_data)):
        if (i+1)%2 == 0:
            finish.append(time_data[i])
        else:
            start.append(time_data[i])
    for i in range(len(start)):
        pairs.append((start[i],finish[i]))
    return (pairs)

def timerange(stim,data_set):
    timedata = []
    with h5py.File(data_set.nwb_file, 'r') as f:
        time = f['stimulus/presentation'][stim + '_stimulus']['timestamps'].value
        if stim == 'spontaneous':
            pairs = segment_spontaneous(time)
        else:
            pairs = segment(time)
        if len(pairs) == 1:
            if stim == 'natural_movie_one':
                timedata.append((stim+data_sets_names[ds],pairs[0]))
            else:
                timedata.append((stim,pairs[0]))
        else:
            for i in range(len(pairs)):
                timedata.append((stim+'_'+str(i+1),pairs[i]))
        return (timedata)
    
def findindex(pairs):
    indexlist = []
    for pair in pairs:
        start, finish = pair[1]
        try: 
            start_index = np.where(timestamps[ds]==start)[0][0]
        except IndexError:
            print ('Error: start index')
        try:
            finish_index = np.where(timestamps[ds]==finish)[0][0]
        except IndexError:
            print ('Error: finish index')
        
        indexlist.append((pair[0],(start_index,finish_index+1)))
    return (indexlist)

def classfix(classes):
    for i in range(len(classes)):
        if classes[i] in [1,2,3]:
            classes[i] = 1
        elif classes[i] in [4,5,6]:
            classes[i] = 2
        elif classes[i] in [7,8,9,10,11,12]:
            classes[i] = 3
        elif classes[i] in [13,14,15]:
            classes[i] = 4
        elif classes[i] in [16,17,18,19]:
            classes[i] = 5
        elif classes[i] in [20,21,22]:
            classes[i] = 6

def create_row(row_number, data):
    row_data = []
    for i in range(len(alpha)):
        stim_epoch = stim_times[legend.index(alpha[i])]
        segments = int(np.floor(len(timestamps[stim_epoch[2]][stim_epoch[1][0]:stim_epoch[1][1]])/301))
        for j in range(segments):
            start = stim_epoch[1][0] + (j*301)
            finish = start + 301
            row_data.append(np.nanmean(data[stim_epoch[2]][start:finish]))
    cell_metrics[row_number] = row_data
    

#### dF/F matrix generation 

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/dff_matrices/'+str(contid)+'.npy'):
        continue
    exps = boc.get_ophys_experiments(experiment_container_ids=[contid])
    for i in range(3):
        if exps[i]['session_type'] == 'three_session_A':
            id1 = exps[i]['id']
        elif exps[i]['session_type'] == 'three_session_B':
            id2 = exps[i]['id']
        else:
            id3 = exps[i]['id']
    dsA = boc.get_ophys_experiment_data(id1)
    dsB = boc.get_ophys_experiment_data(id2)
    dsC = boc.get_ophys_experiment_data(id3)
    data_sets = [dsA,dsB,dsC]
    data_sets_names = ['dsA','dsB','dsC']
    
    cell_list = list(np.intersect1d(dsA.get_cell_specimen_ids(), np.intersect1d(dsB.get_cell_specimen_ids(), dsC.get_cell_specimen_ids())))
    raw_cell_data = []
    for cell in cell_list:
        A_time, A_dff = dsA.get_dff_traces([cell])   
        B_time, B_dff = dsB.get_dff_traces([cell])
        C_time, C_dff = dsC.get_dff_traces([cell])
        
        raw_cell_data.append([cell,[A_dff[0],B_dff[0],C_dff[0]]])
    timestamps = [A_time,B_time,C_time]
    stim_times = []
    iterations = []
    names = []
    
    for ds in range(len(data_sets)):
        data_set = data_sets[ds]
        for stim in all_stimuli:
            if stim == 'locally_sparse_noise_8deg':
                continue
            try:
                pairs = timerange(stim,data_set)
                indices = findindex(pairs)
                if stim not in names:
                    names.append(stim)
                    iterations.append([stim,len(pairs)])
                else:
                    pos = names.index(stim)
                    iterations[pos][1] += len(pairs)
                for i in range(len(indices)):
                    stim_times.append((str(indices[i][0]),indices[i][1],ds))
            except KeyError:
                continue
    timeblocks = 0
    legend = []
    for stim_epoch in stim_times:
        legend.append(stim_epoch[0])
        timeblocks += int(np.floor(len(timestamps[stim_epoch[2]][stim_epoch[1][0]:stim_epoch[1][1]])/301))
    alpha = copy.copy(legend)
    alpha.sort()

    classes = []
    for i in range(len(alpha)):
        stim_epoch = stim_times[legend.index(alpha[i])]
        segments = int(np.floor(len(timestamps[stim_epoch[2]][stim_epoch[1][0]:stim_epoch[1][1]])/301))
        for j in range(segments):
            classes.append(i+1)
    classfix(classes)
        
    cell_metrics = np.zeros((len(cell_list)+1,timeblocks))
    for i in range(len(cell_list)):
        create_row(i,raw_cell_data[i][1])
    cell_metrics[:-1] = stats.zscore(cell_metrics[:-1],axis=1)
    cell_metrics[-1] = classes
    np.save('./dvsc/stimulus_decoding/dff_matrices/'+str(contid),cell_metrics)

### Classification

#### Functions
_For balancing the data to match 80% of the spontaneous stimulus class in the training set and performing a five-fold cross validation to determine the regularization constant that yields the highest accuracy._

In [ ]:
def balance(per_class, dff_matrix): 
    features_train = np.zeros((dff_matrix.shape[0]-1,per_class*6))
    features_test = np.zeros((dff_matrix.shape[0]-1,0))
    classes_train, classes_test = [], []
    classes = dff_matrix[-1]
    for i in range(6):
        label = i + 1
        stim_features = dff_matrix[:-1,np.where(classes==label)[0][0]:np.where(classes==label)[0][-1]+1]
        stim_classes = classes[np.where(classes==label)[0][0]:np.where(classes==label)[0][-1]+1]
        x_train, x_test, y_train, y_test = train_test_split(stim_features.T,stim_classes,train_size=per_class)
        classes_train += list(y_train)
        classes_test += list(y_test)
        features_train[:,per_class*i:per_class*(i+1)] = x_train.T
        features_test = np.concatenate((features_test,x_test.T),axis=1)
    return(features_train, features_test, classes_train, classes_test)

def crossval(classifier,features_train,classes_train,folds):
    c_std = [0.01,0.1,10,100,1000,np.inf]
    fold_size = int(np.floor(features_train.shape[1]/folds))
    extent = folds * fold_size
    leftover = list(np.arange(extent,features_train.shape[1]))
    accuracy = []
    for c_value in c_std:
        yhat = []
        for i in range(folds):
            start = fold_size * i
            finish = fold_size * (i+1)
            x_test = features_train[:,start:finish]
            y_test = classes_train[start:finish]
            x_train = features_train[:,list(np.arange(0,start))+list(np.arange(finish,extent))+leftover]
            y_train = np.asarray(classes_train)[list(np.arange(0,start))+list(np.arange(finish,extent))+leftover]
            if classifier == 'svm':
                clf = svm.LinearSVC(C=c_value)
            elif classifier == 'mlr':
                clf = linear_model.LogisticRegression(C=c_value, multi_class='multinomial', solver='lbfgs')
            clf.fit(x_train.T,y_train)
            yhat += list(clf.predict(x_test.T))
        accuracy.append(accuracy_score(np.asarray(yhat),classes_train[:extent]))
    return(c_std[np.where(accuracy==np.amax(accuracy))[0][0]])


#### Support Vector Machine (SVM)

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/svm_results/accuracy_scores/'+str(contid)+'.npy'):
        continue
    mean_accuracy_scores = []
    confusion_matrices = []
    
    dff_matrix = np.load('./dvsc/stimulus_decoding/dff_matrices/'+str(contid)+'.npy')
    logspace = np.round(np.logspace(0,np.log10(dff_matrix.shape[0]-1),10))
    np.save('./dvsc/stimulus_decoding/sample_size/'+str(contid),logspace)
    timeblocks = dff_matrix.shape[1]
    classes = dff_matrix[-1]
    per_class = int(np.floor(0.8 * ((np.where(classes==5)[0][-1] + 1) - np.where(classes==5)[0][0])))
    
    for sample in logspace:
        subsample_accuracy = []
        subsample_confusion_matrix = np.zeros((6,6))
        
        np.random.shuffle(dff_matrix[:-1])
        subsample_features = dff_matrix[:int(sample)]
        combine = np.zeros((int(sample+1),timeblocks))
        combine[:-1] = subsample_features
        combine[-1] = dff_matrix[-1]
        features_train, features_test, classes_train, classes_test = balance(per_class,combine)
        c_reg = crossval('svm',features_train,classes_train,5)
        
        for trial in range(10):
            np.random.shuffle(dff_matrix[:-1])
            subsample_features = dff_matrix[:int(sample)]
            combine = np.zeros((int(sample+1),timeblocks))
            combine[:-1] = subsample_features
            combine[-1] = dff_matrix[-1]
            features_train, features_test, classes_train, classes_test = balance(per_class,combine)
            clf = svm.LinearSVC(C=c_reg)
            clf.fit(features_train.T,classes_train)
            yhat = clf.predict(features_test.T)
            subsample_accuracy.append(accuracy_score(classes_test,yhat))
            cm = confusion_matrix(classes_test,yhat)
            subsample_confusion_matrix += cm
        mean_accuracy_scores.append(np.mean(subsample_accuracy))
        confusion_matrices.append(subsample_confusion_matrix)
    
    np.save('./dvsc/stimulus_decoding/svm_results/accuracy_scores/'+str(contid),mean_accuracy_scores)
    np.save('./dvsc/stimulus_decoding/svm_results/confusion_matrices/'+str(contid),confusion_matrices)

#### Multinomial Logistic Regression (MLR)

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/mlr_results/accuracy_scores/'+str(contid)+'.npy'):
        continue
    mean_accuracy_scores = []
    confusion_matrices = []
    
    dff_matrix = np.load('./dvsc/stimulus_decoding/dff_matrices/'+str(contid)+'.npy')
    logspace = np.load('./dvsc/stimulus_decoding/sample_size/'+str(contid)+'.npy')
    timeblocks = dff_matrix.shape[1]
    classes = dff_matrix[-1]
    per_class = int(np.floor(0.8 * ((np.where(classes==5)[0][-1] + 1) - np.where(classes==5)[0][0])))
    
    for sample in logspace:
        subsample_accuracy = []
        subsample_confusion_matrix = np.zeros((6,6))
        
        np.random.shuffle(dff_matrix[:-1])
        subsample_features = dff_matrix[:int(sample)]
        combine = np.zeros((int(sample+1),timeblocks))
        combine[:-1] = subsample_features
        combine[-1] = dff_matrix[-1]
        features_train, features_test, classes_train, classes_test = balance(per_class,combine)
        c_reg = crossval('mlr',features_train,classes_train,5)
    
        for trial in range(10):
            np.random.shuffle(dff_matrix[:-1])
            subsample_features = dff_matrix[:int(sample)]
            combine = np.zeros((int(sample+1),timeblocks))
            combine[:-1] = subsample_features
            combine[-1] = dff_matrix[-1]
            features_train, features_test, classes_train, classes_test = balance(per_class,combine)
            clf = linear_model.LogisticRegression(C=c_reg, multi_class='multinomial', solver='lbfgs')
            clf.fit(features_train.T,classes_train)
            yhat = clf.predict(features_test.T)
            subsample_accuracy.append(accuracy_score(classes_test,yhat))
            cm = confusion_matrix(classes_test,yhat)
            subsample_confusion_matrix += cm
        mean_accuracy_scores.append(np.mean(subsample_accuracy))
        confusion_matrices.append(subsample_confusion_matrix)
    
    np.save('./dvsc/stimulus_decoding/mlr_results/accuracy_scores/'+str(contid),mean_accuracy_scores)
    np.save('./dvsc/stimulus_decoding/mlr_results/confusion_matrices/'+str(contid),confusion_matrices)

### Save fitted lines for all experiments

In [ ]:
def func(x,a,b,c):
    return[(1-c)*np.power((1+np.exp(-a*elem)),-b) + c for elem in x]
neuron_levels = [1,2,4,8,16,32,64,128]

##### SVM

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy'):
        continue
    logspace = np.load('./dvsc/stimulus_decoding/sample_size/'+str(contid)+'.npy')
    accuracy_scores = np.load('./dvsc/stimulus_decoding/svm_results/accuracy_scores/'+str(contid)+'.npy')
    interpolated_scores = np.interp(neuron_levels,logspace,accuracy_scores)
    for i in range(len(interpolated_scores)):
        if logspace[-1] < neuron_levels[i]:
            interpolated_scores[i:] = np.nan
    np.save('./dvsc/stimulus_decoding/svm_results/interpolated_scores/'+str(contid),interpolated_scores)
    
    try:
        pos = np.where(np.isnan(interpolated_scores))[0][0]
    except IndexError:
        pos = len(interpolated_scores)
    
    x_data = np.log10(neuron_levels[0:pos])
    y_data = np.asarray(interpolated_scores[0:pos])
    popt, pcov = curve_fit(func,x_data,y_data,bounds=(0,[np.inf,np.inf,1]))
    x_fit = np.logspace(0,np.log10(128))
    y_fit = func(np.log10(x_fit),*popt)
    
    np.save('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid),y_fit)


#### For each stimulus class 

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_0.npy'):
        continue
    logspace = np.load('./dvsc/stimulus_decoding/sample_size/'+str(contid)+'.npy')
    confusion_matrix = np.load('./dvsc/stimulus_decoding/svm_results/confusion_matrices/' + str(contid) + '.npy')
    stim_accuracies = np.zeros((10,6))
    for i in range(10):
        cm = confusion_matrix[i]
        correct = np.zeros(6)
        for j in range(6):
            correct[j] = cm[j,j]
        stim_accuracies[i] = (correct/np.sum(cm,axis=1)) * 100
    for i in range(6):
        accuracy_scores = stim_accuracies[:,i]
        interpolated_scores = np.interp(neuron_levels,logspace,accuracy_scores)
        for j in (range(len(interpolated_scores))):
            if logspace[-1] < neuron_levels[j]:
                interpolated_scores[j:]= np.nan
        try:
            pos = np.where(np.isnan(interpolated_scores))[0][0]
        except IndexError:
            pos = len(interpolated_scores)
        x_data = neuron_levels[0:pos]
        y_data = np.asarray(interpolated_scores[0:pos])/100
        popt, pcov = curve_fit(func,np.log10(x_data),y_data,maxfev=4000,bounds=(0,[np.inf,np.inf,1]))
        x_fit = np.logspace(0,np.log10(128))
        y_fit = func(np.log10(x_fit),*popt)
        np.save('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_'+str(i),y_fit)
        

##### MLR

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/stimulus_decoding/mlr_results/fitted_lines/'+str(contid)+'.npy'):
        continue
    logspace = np.load('./dvsc/stimulus_decoding/sample_size/'+str(contid)+'.npy')
    accuracy_scores = np.load('./dvsc/stimulus_decoding/mlr_results/accuracy_scores/'+str(contid)+'.npy')
    interpolated_scores = np.interp(neuron_levels,logspace,accuracy_scores)
    for i in range(len(interpolated_scores)):
        if logspace[-1] < neuron_levels[i]:
            interpolated_scores[i:] = np.nan
    np.save('./dvsc/stimulus_decoding/mlr_results/interpolated_scores/'+str(contid),interpolated_scores)
    
    try:
        pos = np.where(np.isnan(interpolated_scores))[0][0]
    except IndexError:
        pos = len(interpolated_scores)
    
    x_data = np.log10(neuron_levels[0:pos])
    y_data = np.asarray(interpolated_scores[0:pos])
    popt, pcov = curve_fit(func,x_data,y_data,bounds=(0,[np.inf,np.inf,1]))
    x_fit = np.logspace(0,np.log10(128))
    y_fit = func(np.log10(x_fit),*popt)
    
    np.save('./dvsc/stimulus_decoding/mlr_results/fitted_lines/'+str(contid),y_fit)


Direction Decoding
---
### Create the neural feature vectors
For each mouse, generate a matrix of mean dF/F values for each neuron over 2 second intervals labelled with the corresponding grating direction class.

#### Functions
_For generating the samples._

In [ ]:
def create_row2(row_number, data):
    row_data = []
    for i in range(len(dg_data)):
        start, finish = dg_data[i][1]
        mean = np.nanmean(data[int(start):int(finish)])
        row_data.append(mean)
    for k in range(len(row_data)):
        if row_data[k] != row_data[k]:
            row_data[k] = 0
    return(row_data)

#### dF/F matrix generation

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/direction_decoding/dff_matrices/'+str(contid)+'.npy'):
        continue
    exps = boc.get_ophys_experiments(experiment_container_ids=[contid])
    for i in range(3):
        if exps[i]['session_type'] == 'three_session_A':
            id1 = exps[i]['id']
        elif exps[i]['session_type'] == 'three_session_B':
            id2 = exps[i]['id']
        else:
            id3 = exps[i]['id']
    dsA = boc.get_ophys_experiment_data(id1)
    cell_list = dsA.get_cell_specimen_ids()
    raw_cell_data = []
    for cell in cell_list:
        A_time, A_dff = dsA.get_dff_traces([cell])   
        raw_cell_data.append([cell,A_dff,A_time])
    
    dg_table = dsA.get_stimulus_table('drifting_gratings')
    dg_data = []
    for i in range(len(dg_table)):
        data = dg_table.iloc[i]
        if data[2] == 1:
            continue
        dg_data.append([data[1],[data[3],data[4]]])
        
    timeblocks = len(dg_data)
    classes = []
    for i in range(len(dg_data)):
        classes.append(dg_data[i][0])
    
    cell_metrics = np.zeros((len(cell_list)+1,timeblocks))
    for i in range(len(cell_list)):
        cell_metrics[i] = create_row2(i,raw_cell_data[i][1][0])
    cell_metrics[:-1] = stats.zscore(cell_metrics[:-1],axis=1)
    cell_metrics[-1] = classes
    np.save('./dvsc/direction_decoding/dff_matrices/'+str(contid),cell_metrics)

### Classification

#### Support Vector Machines (SVM)

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/direction_decoding/svm_results/accuracy_scores/'+str(contid)+'.npy'):
        continue
    mean_accuracy_scores = []
    confusion_matrices = []
    
    dff_matrix = np.load('./dvsc/direction_decoding/dff_matrices/'+str(contid)+'.npy')
    logspace = np.round(np.logspace(0,np.log10(dff_matrix.shape[0]-1),10))
    np.save('./dvsc/direction_decoding/sample_size/'+str(contid),logspace)
    timeblocks = dff_matrix.shape[1]
    classes = dff_matrix[-1]
    
    for sample in logspace:
        subsample_accuracy = []
        subsample_confusion_matrix = np.zeros((8,8))
        
        np.random.shuffle(dff_matrix[:-1])
        subsample_features = dff_matrix[:int(sample)]
        features_train, features_test, classes_train, classes_test = train_test_split(subsample_features.T,dff_matrix[-1],train_size=0.8)
        c_reg = crossval('svm',features_train.T,classes_train,5)
        
        for trial in range(10):
            np.random.shuffle(dff_matrix[:-1])
            subsample_features = dff_matrix[:int(sample)]
            features_train, features_test, classes_train, classes_test = train_test_split(subsample_features.T,dff_matrix[-1],train_size=0.8)
            clf = svm.LinearSVC(C=c_reg)
            clf.fit(features_train,classes_train)
            yhat = clf.predict(features_test)
            subsample_accuracy.append(accuracy_score(classes_test,yhat))
            cm = confusion_matrix(classes_test,yhat)
            subsample_confusion_matrix += cm
        mean_accuracy_scores.append(np.mean(subsample_accuracy))
        confusion_matrices.append(subsample_confusion_matrix)
    
    np.save('./dvsc/direction_decoding/svm_results/accuracy_scores/'+str(contid),mean_accuracy_scores)
    np.save('./dvsc/direction_decoding/svm_results/confusion_matrices/'+str(contid),confusion_matrices)

#### Multinomial Logistic Regression (MLR)

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/direction_decoding/mlr_results/accuracy_scores/'+str(contid)+'.npy'):
        continue
    mean_accuracy_scores = []
    confusion_matrices = []
    
    dff_matrix = np.load('./dvsc/direction_decoding/dff_matrices/'+str(contid)+'.npy')
    logspace = np.load('./dvsc/direction_decoding/sample_size/'+str(contid)+'.npy')
    timeblocks = dff_matrix.shape[1]
    classes = dff_matrix[-1]
    
    for sample in logspace:
        subsample_accuracy = []
        subsample_confusion_matrix = np.zeros((8,8))
        
        np.random.shuffle(dff_matrix[:-1])
        subsample_features = dff_matrix[:int(sample)]
        features_train, features_test, classes_train, classes_test = train_test_split(subsample_features.T,dff_matrix[-1],train_size=0.8)
        c_reg = crossval('mlr',features_train.T,classes_train,5)
        
        for trial in range(10):
            np.random.shuffle(dff_matrix[:-1])
            subsample_features = dff_matrix[:int(sample)]
            features_train, features_test, classes_train, classes_test = train_test_split(subsample_features.T,dff_matrix[-1],train_size=0.8)
            clf = linear_model.LogisticRegression(C=c_reg, multi_class='multinomial', solver='lbfgs')
            clf.fit(features_train,classes_train)
            yhat = clf.predict(features_test)
            subsample_accuracy.append(accuracy_score(classes_test,yhat))
            cm = confusion_matrix(classes_test,yhat)
            subsample_confusion_matrix += cm
        mean_accuracy_scores.append(np.mean(subsample_accuracy))
        confusion_matrices.append(subsample_confusion_matrix)
    
    np.save('./dvsc/direction_decoding/mlr_results/accuracy_scores/'+str(contid),mean_accuracy_scores)
    np.save('./dvsc/direction_decoding/mlr_results/confusion_matrices/'+str(contid),confusion_matrices)

### Save interpolated and fitted lines for all experiments

##### SVM

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy'):
        continue
    logspace = np.load('./dvsc/direction_decoding/sample_size/'+str(contid)+'.npy')
    accuracy_scores = np.load('./dvsc/direction_decoding/svm_results/accuracy_scores/'+str(contid)+'.npy')
    interpolated_scores = np.interp(neuron_levels,logspace,accuracy_scores)
    for i in range(len(interpolated_scores)):
        if logspace[-1] < neuron_levels[i]:
            interpolated_scores[i:] = np.nan
    np.save('./dvsc/direction_decoding/svm_results/interpolated_scores/'+str(contid),interpolated_scores)
    
    try:
        pos = np.where(np.isnan(interpolated_scores))[0][0]
    except IndexError:
        pos = len(interpolated_scores)
    
    x_data = np.log10(neuron_levels[0:pos])
    y_data = np.asarray(interpolated_scores[0:pos])
    popt, pcov = curve_fit(func,x_data,y_data,bounds=(0,[np.inf,np.inf,1]))
    x_fit = np.logspace(0,np.log10(128))
    y_fit = func(np.log10(x_fit),*popt)
    
    np.save('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid),y_fit)


##### MLR

In [ ]:
for contid in all_ecs_ids:
    if os.path.exists('./dvsc/direction_decoding/mlr_results/fitted_lines/'+str(contid)+'.npy'):
        continue
    logspace = np.load('./dvsc/direction_decoding/sample_size/'+str(contid)+'.npy')
    accuracy_scores = np.load('./dvsc/direction_decoding/mlr_results/accuracy_scores/'+str(contid)+'.npy')
    interpolated_scores = np.interp(neuron_levels,logspace,accuracy_scores)
    for i in range(len(interpolated_scores)):
        if logspace[-1] < neuron_levels[i]:
            interpolated_scores[i:] = np.nan
    np.save('./dvsc/direction_decoding/mlr_results/interpolated_scores/'+str(contid),interpolated_scores)
    
    try:
        pos = np.where(np.isnan(interpolated_scores))[0][0]
    except IndexError:
        pos = len(interpolated_scores)
    
    x_data = np.log10(neuron_levels[0:pos])
    y_data = np.asarray(interpolated_scores[0:pos])
    popt, pcov = curve_fit(func,x_data,y_data,bounds=(0,[np.inf,np.inf,1]))
    x_fit = np.logspace(0,np.log10(128))
    y_fit = func(np.log10(x_fit),*popt)
    
    np.save('./dvsc/direction_decoding/mlr_results/fitted_lines/'+str(contid),y_fit)


Results
---
#### Sort experiment containers by the cortical area and imaging depth
* six cortical areas
* ten depths sorted into four groups 

In [ ]:
areas = boc.get_all_targeted_structures()
depths = boc.get_all_imaging_depths()

if os.path.exists('./dvsc/ids/groupids.npy'):
    groupids = np.load('./dvsc/ids/groupids.npy')
    
    area_ids = groupids[0]
    visal = area_ids[areas.index('VISal')]
    visam = area_ids[areas.index('VISam')]
    visl = area_ids[areas.index('VISl')]
    visp = area_ids[areas.index('VISp')]
    vispm = area_ids[areas.index('VISpm')]
    visrl = area_ids[areas.index('VISrl')]
    
    depth_ids = groupids[1]
    dg1 = depth_ids[depths.index(175)]
    dg2 = depth_ids[depths.index(265)] + depth_ids[depths.index(275)] + depth_ids[depths.index(300)]
    dg3 = depth_ids[depths.index(325)] + depth_ids[depths.index(335)] + depth_ids[depths.index(350)]
    dg4 = depth_ids[depths.index(365)] + depth_ids[depths.index(375)] + depth_ids[depths.index(435)]  

else: 
    groupids = []
    area_ids = [[],[],[],[],[],[]]
    depth_ids = [[],[],[],[],[],[],[],[],[],[],[]]
    for contid in all_ecs_ids:
        exp = boc.get_ophys_experiments(experiment_container_ids=[contid])[0]
        area_ids[areas.index(exp['targeted_structure'])].append(contid)
        depth_ids[depths.index(exp['imaging_depth'])].append(contid)
    groupids.append(area_ids)
    groupids.append(depth_ids)
    np.save('./dvsc/ids/groupids.npy',groupids)
    
    area_ids = groupids[0]
    visal = area_ids[areas.index('VISal')]
    visam = area_ids[areas.index('VISam')]
    visl = area_ids[areas.index('VISl')]
    visp = area_ids[areas.index('VISp')]
    vispm = area_ids[areas.index('VISpm')]
    visrl = area_ids[areas.index('VISrl')]
    
    depth_ids = groupids[1]
    dg1 = depth_ids[depths.index(175)]
    dg2 = depth_ids[depths.index(265)] + depth_ids[depths.index(275)] + depth_ids[depths.index(300)]
    dg3 = depth_ids[depths.index(325)] + depth_ids[depths.index(335)] + depth_ids[depths.index(350)]
    dg4 = depth_ids[depths.index(365)] + depth_ids[depths.index(375)] + depth_ids[depths.index(435)]  


#### Formatting details
_Assigning colors to visual areas, imaging depths, and stimuli. Creating a function to easily produce legends._

In [ ]:
labels = ['{}'.format(x) for x in neuron_levels]

plt.rcParams['pdf.fonttype'] = 42 

red = '#AD160D' 
orange = '#E46307' 
yellow = '#C4A00E'
green = '#9BC45C'
blue = '#5C7BC4'
purple = '#8C5CC4'

cortical_areas = [visp, vispm, visl, visal, visam, visrl]
area_colors = [red, orange, green, blue, purple, yellow]
area_names = ['VISp','VISpm','VISl','VISal','VISam','VISrl']

imaging_depths = [dg1, dg2, dg3, dg4]
depth_colors = [red, blue, orange, purple]
depth_names = ['175 µm','265, 275, 300 µm','325, 335, 350 µm','365, 375, 435 µm']

colors_dict = {'dg':red,'sg':orange,'lsn':yellow,'nm':green,'ns':blue,'spon':purple}
stim_colors = [red, yellow, green, blue, purple, orange]
stim_names = ['drifting gratings','locally sparse noise','natural movies','natural scenes','spontaneous','static gratings']

fs = 9

def legend(subplot,feature):
    x = [0]
    if feature == 'area':
        categ = cortical_areas
        subplot.plot(x,x,c=red)
        subplot.plot(x,x,c=orange)
        subplot.plot(x,x,c=green)
        subplot.plot(x,x,c=blue)
        subplot.plot(x,x,c=purple)
        subplot.plot(x,x,c=yellow)
        subplot.legend(['VISp'+ ' (' + str(len(categ[0])) + ')','VISpm'+ ' (' + str(len(categ[1])) + ')','VISl'+ ' (' + str(len(categ[2])) + ')','VISal'+ ' (' + str(len(categ[3])) + ')','VISam'+ ' (' + str(len(categ[4])) + ')','VISrl'+ ' (' + str(len(categ[5])) + ')'], loc = 6,fontsize =fs)
    if feature == 'depth':
        categ = imaging_depths
        subplot.plot(x,x,c=red)
        subplot.plot(x,x,c=blue)
        subplot.plot(x,x,c=orange)
        subplot.plot(x,x,c=purple)
        subplot.legend(['175 µm (' + str(len(categ[0])) + ')','265, 275, 300 µm (' + str(len(categ[1])) + ')','325, 335, 350 µm (' + str(len(categ[2])) + ')','365, 375, 435 µm (' + str(len(categ[3])) + ')'], loc = 6,fontsize =fs)
    if feature == 'stimuli':
        subplot.plot(x,x,c=colors_dict['lsn'], label = 'locally sparse noise')
        subplot.plot(x,x,c=colors_dict['sg'], label ='static gratings')
        subplot.plot(x,x,c=colors_dict['dg'], label = 'drifting gratings')
        subplot.plot(x,x,c=colors_dict['ns'], label ='natural scenes')
        subplot.plot(x,x,c=colors_dict['spon'], label = 'spontaneous')
        subplot.plot(x,x,c=colors_dict['nm'], label ='natural movies')
        subplot.legend(loc = 6, fontsize = fs)

    plt.axis('off')

## By visual area

**Stimulus Category Decoding** – ***Interpolated points (black dots) with curve fits (solid lines)*** 

In [ ]:
unit = int(len(cortical_areas))
fig = plt.figure(figsize=(int(unit),4),dpi=400) 
gs = gridspec.GridSpec(2,int(unit/2))

for k in range(len(cortical_areas)):
    id_list = cortical_areas[k]
    acc = fig.add_subplot(gs[int(k/(unit/2)),int(k%(unit/2))])
    acc.set_ylim(0,100)
    if k >= (unit/2):
        plt.xlabel('neurons',fontsize=fs)
    if k == 0 or k == int(unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-1)
                 
    for j in range(len(id_list)):
        contid = id_list[j]
        col = area_colors[k]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy') * 100
        plt.plot(np.log10(np.logspace(0,np.log10(128))), y_fit, c = col, lw = 1, alpha = 0.4, zorder = 1)
        interpolated_points = np.load('./dvsc/stimulus_decoding/svm_results/interpolated_scores/'+str(contid)+'.npy') * 100
        plt.scatter(np.log10([1,2,4,8,16,32,64,128])+(np.random.rand(8)/15), interpolated_points, s = 1, color = col, alpha = 0.7, edgecolor = 'k', zorder = 2)
    acc.set_title(area_names[k] + ' (n = ' + str(len(cortical_areas[k])) + ')' ,fontsize=fs)
    plt.axhline(100/6, c = 'k', alpha = 0.5, lw = 0.8)
    plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/stimulus_decoding_visual_area.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Category Decoding** –
***Population averaged accuracy by visual area*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
fig = plt.figure(figsize = (3,2), dpi = 400)
gs = gridspec.GridSpec(1,4)
leg = fig.add_subplot(gs[:,3])
legend(leg,'area')
acc = fig.add_subplot(gs[0:2,0:3])
plt.xlabel('neurons', fontsize = fs)
plt.ylabel('classification accuracy (%)', fontsize = fs)
acc.set_ylim(0,100)

for k in range(len(cortical_areas)):
    area = cortical_areas[k]
    accuracy = np.zeros((len(area),len(np.logspace(0,np.log(128)))))
    for i in range(len(area)):
        contid = area[i]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        accuracy[i] = y_fit
    average = np.nanmean(accuracy,axis=0)*100
    error = np.nanstd(accuracy*100,axis=0)/np.sqrt(len(area))
    col = area_colors[k]
    plt.plot(np.log10(np.logspace(0,np.log10(128))), average, color = col, lw = 1)
    plt.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
plt.yticks(fontsize = fs-2)
acc.set_title('Accuracy by Area', fontsize = fs)
plt.axhline(100/6, c = 'k', alpha = 0.5, lw = 0.8)
plt.text(np.log10(36),(100/6)+2,'chance', fontsize = fs-1, color = 'k', alpha = 0.7)
plt.savefig('./dvsc/figures/stimulus_decoding_visual_area_average.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Category Decoding** –***Tukey's Test*** _Significance Plot (p < 0.05)_

In [ ]:
points, groups = [], []
names = area_names
for k in range(len(cortical_areas)):
    id_list = cortical_areas[k]
    n128 = []
    group_label = []
    for contid in id_list:
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        n128.append(y_fit[-1])
        group_label.append(names[k])
    points.append(n128)
    groups.append(group_label)
points = np.concatenate(points)
groups = np.concatenate(groups)

tukey_results = pairwise_tukeyhsd(points,groups)
summary = tukey_results.summary()
x_axis = []
y_axis = []
x_axis2 = []
y_axis2 = []
for i in range(1,len(summary)):
    if str(summary[i][5]) == 'True':
        x_axis.append(names.index(str(summary[i][0])))
        y_axis.append(np.flip(names,axis=0).tolist().index(str(summary[i][1])))
        x_axis2.append(np.flip(names,axis=0).tolist().index(str(summary[i][0])))
        y_axis2.append(names.index(str(summary[i][1])))
    
plt.figure(figsize=(2.5,2.5),dpi=400)
plt.scatter(x_axis,y_axis,marker = (6,2,0), c='k')
plt.scatter(y_axis2, x_axis2,marker = (6,2,0), c='k')
plt.title('significance (p<0.05)')
plt.xticks([0,1,2,3,4,5],names,rotation=90, fontsize = fs)
plt.yticks([0,1,2,3,4,5],np.flip(names,axis=0), fontsize = fs)
plt.xlim(-0.5,len(names)-0.5)
plt.ylim(-0.5,len(names)-0.5)
plt.savefig('./dvsc/figures/stimulus_decoding_visual_area_significance.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()


**Direction Decoding** – ***Interpolated points*** *(black dots)* ***with curve fits*** *(solid lines)* 

In [ ]:
unit = int(len(cortical_areas))
fig = plt.figure(figsize=(int(unit),4),dpi=400) 
gs = gridspec.GridSpec(2,int(unit/2))

for k in range(len(cortical_areas)):
    id_list = cortical_areas[k]
    acc = fig.add_subplot(gs[int(k/(unit/2)),int(k%(unit/2))])
    acc.set_ylim(0,100)
    if k >= (unit/2):
        plt.xlabel('neurons',fontsize=fs)
    if k == 0 or k == int(unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-1)
                 
    for j in range(len(id_list)):
        contid = id_list[j]
        col = area_colors[k]
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy') * 100
        plt.plot(np.log10(np.logspace(0,np.log10(128))), y_fit, c = col, lw = 1, alpha = 0.4, zorder = 1)
        interpolated_points = np.load('./dvsc/direction_decoding/svm_results/interpolated_scores/'+str(contid)+'.npy') * 100
        plt.scatter(np.log10([1,2,4,8,16,32,64,128])+(np.random.rand(8)/15), interpolated_points, s = 1, color = col, alpha = 0.7, edgecolor = 'k', zorder = 2)
    acc.set_title(area_names[k] + ' (n = ' + str(len(cortical_areas[k])) + ')' ,fontsize=fs)
    plt.axhline(100/8, c = 'k', alpha = 0.5, lw = 0.8)
    plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/direction_decoding_visual_area.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Direction Decoding** –
***Population averaged accuracy by visual area*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
fig = plt.figure(figsize = (3,2), dpi = 400)
gs = gridspec.GridSpec(1,4)
leg = fig.add_subplot(gs[:,3])
legend(leg,'area')
acc = fig.add_subplot(gs[0:2,0:3])
plt.xlabel('neurons', fontsize = fs)
plt.ylabel('classification accuracy (%)', fontsize = fs)
acc.set_ylim(0,100)

for k in range(len(cortical_areas)):
    area = cortical_areas[k]
    accuracy = np.zeros((len(area),len(np.logspace(0,np.log(128)))))
    for i in range(len(area)):
        contid = area[i]
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        accuracy[i] = y_fit
    average = np.nanmean(accuracy,axis=0)*100
    error = np.nanstd(accuracy*100,axis=0)/np.sqrt(len(area))
    col = area_colors[k]
    plt.plot(np.log10(np.logspace(0,np.log10(128))), average, color = col, lw = 1)
    plt.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
plt.yticks(fontsize = fs-2)
acc.set_title('Accuracy by Area', fontsize = fs)
plt.axhline(100/8, c = 'k', alpha = 0.5, lw = 0.8)
plt.text(np.log10(36),(100/8)+2,'chance', fontsize = fs-1, color = 'k', alpha = 0.7)
plt.savefig('./dvsc/figures/direction_decoding_visual_area_average.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Direction Decoding** –***Tukey's Test*** _Significance Plot (p < 0.05)_

In [ ]:
points, groups = [], []
names = area_names
for k in range(len(cortical_areas)):
    id_list = cortical_areas[k]
    n128 = []
    group_label = []
    for contid in id_list:
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        n128.append(y_fit[-1])
        group_label.append(names[k])
    points.append(n128)
    groups.append(group_label)
points = np.concatenate(points)
groups = np.concatenate(groups)

tukey_results = pairwise_tukeyhsd(points,groups)
summary = tukey_results.summary()
x_axis = []
y_axis = []
x_axis2 = []
y_axis2 = []
for i in range(1,len(summary)):
    if str(summary[i][5]) == 'True':
        x_axis.append(names.index(str(summary[i][0])))
        y_axis.append(np.flip(names,axis=0).tolist().index(str(summary[i][1])))
        x_axis2.append(np.flip(names,axis=0).tolist().index(str(summary[i][0])))
        y_axis2.append(names.index(str(summary[i][1])))
    
plt.figure(figsize=(2.5,2.5),dpi=400)
plt.scatter(x_axis,y_axis,marker = (6,2,0), c='k')
plt.scatter(y_axis2, x_axis2,marker = (6,2,0), c='k')
plt.title('significance (p<0.05)')
plt.xticks([0,1,2,3,4,5],names,rotation=90, fontsize = fs)
plt.yticks([0,1,2,3,4,5],np.flip(names,axis=0), fontsize = fs)
plt.xlim(-0.5,len(names)-0.5)
plt.ylim(-0.5,len(names)-0.5)
plt.savefig('./dvsc/figures/direction_decoding_visual_area_significance.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

## By imaging depth 

**Stimulus Category Decoding** –
***Interpolated points (black dots) with curve fits (solid lines)***

In [ ]:
unit = int(len(imaging_depths))
fig = plt.figure(figsize=(int(unit),4),dpi=400) 
gs = gridspec.GridSpec(2,int(unit/2))

for k in range(len(imaging_depths)):
    id_list = imaging_depths[k]
    acc = fig.add_subplot(gs[int(k/(unit/2)),int(k%(unit/2))])
    acc.set_ylim(0,100)
    if k >= (unit/2):
        plt.xlabel('neurons',fontsize=fs)
    if k == 0 or k == int(unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-1)
                 
    for j in range(len(id_list)): # for each id in the category
        contid = id_list[j]
        col = depth_colors[k]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy') * 100
        plt.plot(np.log10(np.logspace(0,np.log10(128))), y_fit, c = col, lw = 1, alpha = 0.4, zorder = 1)
        interpolated_points = np.load('./dvsc/stimulus_decoding/svm_results/interpolated_scores/'+str(contid)+'.npy') * 100
        plt.scatter(np.log10([1,2,4,8,16,32,64,128])+(np.random.rand(8)/15), interpolated_points, s = 1, color = col, alpha = 0.7, edgecolor = 'k', zorder = 2)
    acc.set_title(depth_names[k] + ' (n = ' + str(len(imaging_depths[k])) + ')' ,fontsize=fs-2.5)
    plt.axhline(100/6, c = 'k', alpha = 0.5, lw = 0.8)
    plt.xticks(np.log10(neuron_levels), labels, fontsize=fs-3)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/stimulus_decoding_imaging_depth.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Category Decoding** –
***Population averaged accuracy by imaging depth*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
fig = plt.figure(figsize = (3,2), dpi = 400)
gs = gridspec.GridSpec(1,4)
leg = fig.add_subplot(gs[:,3])
legend(leg,'depth')
acc = fig.add_subplot(gs[0:2,0:3])
plt.xlabel('neurons', fontsize = fs)
plt.ylabel('classification accuracy (%)', fontsize = fs)
acc.set_ylim(0,100)

for k in range(len(imaging_depths)):
    depth = imaging_depths[k]
    accuracy = np.zeros((len(depth),len(np.logspace(0,np.log(128)))))
    for i in range(len(depth)):
        contid = depth[i]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        accuracy[i] = y_fit
    average = np.nanmean(accuracy,axis=0)*100
    error = np.nanstd(accuracy*100,axis=0)/np.sqrt(len(area))
    col = depth_colors[k]
    plt.plot(np.log10(np.logspace(0,np.log10(128))), average, color = col, lw = 1)
    plt.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
plt.yticks(fontsize = fs-2)
acc.set_title('Accuracy by Depth', fontsize = fs)
plt.axhline(100/6, c = 'k', alpha = 0.5, lw = 0.8)
plt.text(np.log10(36),(100/6)+2,'chance', fontsize = fs-1, color = 'k', alpha = 0.7)
plt.savefig('./dvsc/figures/stimulus_decoding_imaging_depth_average.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Category Decoding** –***Tukey's Test*** _Significance Plot (p < 0.05)_

In [ ]:
points, groups = [], []
names = depth_names
for k in range(len(imaging_depths)):
    id_list = imaging_depths[k]
    n128 = []
    group_label = []
    for contid in id_list:
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        n128.append(y_fit[-1])
        group_label.append(names[k])
    points.append(n128)
    groups.append(group_label)
points = np.concatenate(points)
groups = np.concatenate(groups)

tukey_results = pairwise_tukeyhsd(points,groups)
summary = tukey_results.summary()
x_axis = []
y_axis = []
x_axis2 = []
y_axis2 = []
for i in range(1,len(summary)):
    if str(summary[i][5]) == 'True':
        x_axis.append(names.index(str(summary[i][0])))
        y_axis.append(np.flip(names,axis=0).tolist().index(str(summary[i][1])))
        x_axis2.append(np.flip(names,axis=0).tolist().index(str(summary[i][0])))
        y_axis2.append(names.index(str(summary[i][1])))
    
plt.figure(figsize=(2.5,2.5),dpi=400)
plt.scatter(x_axis,y_axis,marker = (6,2,0), c='k')
plt.scatter(y_axis2, x_axis2,marker = (6,2,0), c='k')
plt.title('significance (p<0.05)')
plt.xticks([0,1,2,3,4,5],names,rotation=90, fontsize = fs)
plt.yticks([0,1,2,3,4,5],np.flip(names,axis=0), fontsize = fs)
plt.xlim(-0.5,len(names)-0.5)
plt.ylim(-0.5,len(names)-0.5)
plt.savefig('./dvsc/figures/stimulus_decoding_imaging_depth_significance.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Direction Decoding** –
***Interpolated points (black dots) with curve fits (solid lines)***

In [ ]:
unit = int(len(imaging_depths))
fig = plt.figure(figsize=(int(unit),4),dpi=400) 
gs = gridspec.GridSpec(2,int(unit/2))

for k in range(len(imaging_depths)):
    id_list = imaging_depths[k]
    acc = fig.add_subplot(gs[int(k/(unit/2)),int(k%(unit/2))])
    acc.set_ylim(0,100)
    if k >= (unit/2):
        plt.xlabel('neurons',fontsize=fs)
    if k == 0 or k == int(unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-1)
                 
    for j in range(len(id_list)): # for each id in the category
        contid = id_list[j]
        col = depth_colors[k]
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy') * 100
        plt.plot(np.log10(np.logspace(0,np.log10(128))), y_fit, c = col, lw = 1, alpha = 0.4, zorder = 1)
        interpolated_points = np.load('./dvsc/direction_decoding/svm_results/interpolated_scores/'+str(contid)+'.npy') * 100
        plt.scatter(np.log10([1,2,4,8,16,32,64,128])+(np.random.rand(8)/15), interpolated_points, s = 1, color = col, alpha = 0.7, edgecolor = 'k', zorder = 2)
    acc.set_title(depth_names[k] + ' (n = ' + str(len(imaging_depths[k])) + ')' ,fontsize=fs-2.5)
    plt.axhline(100/8, c = 'k', alpha = 0.5, lw = 0.8)
    plt.xticks(np.log10(neuron_levels), labels, fontsize=fs-3)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/direction_decoding_imaging_depth.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Direction Decoding** –
***Population averaged accuracy by imaging depth*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
fig = plt.figure(figsize = (3,2), dpi = 400)
gs = gridspec.GridSpec(1,4)
leg = fig.add_subplot(gs[:,3])
legend(leg,'depth')
acc = fig.add_subplot(gs[0:2,0:3])
plt.xlabel('neurons', fontsize = fs)
plt.ylabel('classification accuracy (%)', fontsize = fs)
acc.set_ylim(0,100)

for k in range(len(imaging_depths)):
    depth = imaging_depths[k]
    accuracy = np.zeros((len(depth),len(np.logspace(0,np.log(128)))))
    for i in range(len(depth)):
        contid = depth[i]
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        accuracy[i] = y_fit
    average = np.nanmean(accuracy,axis=0)*100
    error = np.nanstd(accuracy*100,axis=0)/np.sqrt(len(area))
    col = depth_colors[k]
    plt.plot(np.log10(np.logspace(0,np.log10(128))), average, color = col, lw = 1)
    plt.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
plt.yticks(fontsize = fs-2)
acc.set_title('Accuracy by Depth', fontsize = fs)
plt.axhline(100/8, c = 'k', alpha = 0.5, lw = 0.8)
plt.text(np.log10(36),(100/6)+2,'chance', fontsize = fs-1, color = 'k', alpha = 0.7)
plt.savefig('./dvsc/figures/direction_decoding_imaging_depth_average.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Direction Decoding** –***Tukey's Test*** _Significance Plot (p < 0.05)_

In [ ]:
points, groups = [], []
names = depth_names
for k in range(len(imaging_depths)):
    id_list = imaging_depths[k]
    n128 = []
    group_label = []
    for contid in id_list:
        y_fit = np.load('./dvsc/direction_decoding/svm_results/fitted_lines/'+str(contid)+'.npy')
        n128.append(y_fit[-1])
        group_label.append(names[k])
    points.append(n128)
    groups.append(group_label)
points = np.concatenate(points)
groups = np.concatenate(groups)

tukey_results = pairwise_tukeyhsd(points,groups)
summary = tukey_results.summary()
x_axis = []
y_axis = []
x_axis2 = []
y_axis2 = []
for i in range(1,len(summary)):
    if str(summary[i][5]) == 'True':
        x_axis.append(names.index(str(summary[i][0])))
        y_axis.append(np.flip(names,axis=0).tolist().index(str(summary[i][1])))
        x_axis2.append(np.flip(names,axis=0).tolist().index(str(summary[i][0])))
        y_axis2.append(names.index(str(summary[i][1])))
    
plt.figure(figsize=(2.5,2.5),dpi=400)
plt.scatter(x_axis,y_axis,marker = (6,2,0), c='k')
plt.scatter(y_axis2, x_axis2,marker = (6,2,0), c='k')
plt.title('significance (p<0.05)')
plt.xticks([0,1,2,3,4,5],names,rotation=90, fontsize = fs)
plt.yticks([0,1,2,3,4,5],np.flip(names,axis=0), fontsize = fs)
plt.xlim(-0.5,len(names)-0.5)
plt.ylim(-0.5,len(names)-0.5)
plt.savefig('./dvsc/figures/direction_decoding_imaging_depth_significance.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

## By stimulus category

**Stimulus Decoding** –
***Stimulus averaged accuracy*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
fig = plt.figure(figsize=(3,2),dpi=400)
gs = gridspec.GridSpec(1,4)
leg = fig.add_subplot(gs[:,3])
legend(leg,'stimuli')
acc = fig.add_subplot(gs[:,:3])
acc.set_ylim(0,100)
plt.xlabel('neurons', fontsize = fs)
plt.ylabel('classification accuracy (%)', fontsize = fs)

for k in range(6): # for each category
    accuracy = np.zeros((186,len(np.logspace(0,np.log10(128)))))
    for j in range(186): 
        contid = all_ecs_ids[j]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_'+str(k)+'.npy')
        accuracy[j] = y_fit
    average = np.nanmean(accuracy*100, axis = 0)
    error = np.nanstd(accuracy*100,axis=0)/np.sqrt(186)
    col = stim_colors[k]
    plt.plot(np.log10(np.logspace(0,np.log10(128))), average, c = col, lw = 1)
    plt.fill_between(np.log10(np.logspace(0,np.log10(128))),average-error,average+error,facecolor=col,alpha=0.4)
acc.set_title('Accuracy by Stimulus',fontsize=fs)
plt.xticks(np.log10(neuron_levels), labels, fontsize = fs-2)
plt.yticks(fontsize = fs-2)
plt.savefig('./dvsc/figures/stimulus_decoding_stimuli.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Decoding** –
***Tukey Range Test*** _Significance Plot (p < 0.05)_

In [ ]:
points = []
groups = []
for k in range(6):
    n128 = []
    for j in range(186): # for each id in the category
        contid = all_ecs_ids[j]
        y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_'+str(k)+'.npy')
        n128.append(y_fit[-1])
        groups.append(stim_names[k])
    points.append(n128)
points = np.concatenate(points)

tukey_results = pairwise_tukeyhsd(points,groups, alpha=0.05)
summary = tukey_results.summary()

x_axis = []
y_axis = []
x_axis2 = []
y_axis2 = []
for i in range(1,len(summary)):
    if str(summary[i][5]) == 'True':
        x_axis.append(stim_names.index(str(summary[i][0])))
        y_axis.append(np.flip(stim_names,axis=0).tolist().index(str(summary[i][1])))
        x_axis2.append(np.flip(stim_names,axis=0).tolist().index(str(summary[i][0])))
        y_axis2.append(stim_names.index(str(summary[i][1])))
fig = plt.figure(figsize=(2.5,2.5),dpi=400)
plt.scatter(x_axis,y_axis,marker = (6,2,0),c='k')
plt.scatter(y_axis2, x_axis2,marker = (6,2,0),c='k')
plt.title('significance (p<0.05)')
plt.xticks([0,1,2,3,4,5],stim_names,rotation=90,fontsize=8)
plt.yticks([0,1,2,3,4,5],np.flip(stim_names,axis=0),fontsize=8)
plt.xlim(-0.5,5.5)
plt.ylim(-0.5,5.5)
plt.savefig('./dvsc/figures/stimulus_decoding_stimuli_significance.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Decoding** –
***Stimulus averaged accuracy across visual areas*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
unit = int(len(cortical_areas))
fig = plt.figure(figsize=(int(unit),4),dpi=400)
gs = gridspec.GridSpec(2,int(unit/2))

for w in range(len(cortical_areas)):
    contid_set = cortical_areas[w]
    acc = fig.add_subplot(gs[int(w/(unit/2)),int(w%(unit/2))]) # accuracy rate plot
    acc.set_ylim(0,100)
    if w >= (unit/2):
        plt.xlabel('neurons',fontsize=fs) 
    if w == 0 or w == (unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-2)

    for k in range(6):
        accuracy = np.zeros((len(contid_set),len(np.logspace(0,np.log10(128)))))
        for j in range(len(contid_set)): 
            contid = contid_set[j]
            y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_'+str(k)+'.npy')
            accuracy[j] = y_fit
        average = np.nanmean(accuracy * 100, axis = 0)
        error = np.nanstd(accuracy * 100, axis = 0)/(np.sqrt(len(contid_set)))
        col = stim_colors[k]
        acc.plot(np.log10(np.logspace(0,np.log10(128))), average, c = col, lw = 1)
        acc.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
    acc.set_title(area_names[w]+ ' (n = ' + str(len(cortical_areas[k])) + ')' ,fontsize=fs)
    plt.xticks(np.log10(neuron_levels),labels,fontsize=fs-2)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/stimulus_decoding_visual_area_stimuli.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

**Stimulus Decoding** –
***Stimulus averaged accuracy by imaging depth*** *(solid lines)* ***with standard error*** *(shaded regions)*

In [ ]:
unit = int(len(imaging_depths))
fig = plt.figure(figsize=(int(unit),4),dpi=400)
gs = gridspec.GridSpec(2,int(unit/2))

for w in range(len(imaging_depths)):
    contid_set = imaging_depths[w]
    acc = fig.add_subplot(gs[int(w/(unit/2)),int(w%(unit/2))]) # accuracy rate plot
    acc.set_ylim(0,100)
    if w >= (unit/2):
        plt.xlabel('neurons',fontsize=fs) 
    if w == 0 or w == (unit/2):
        plt.ylabel('classification accuracy (%)',fontsize=fs-2)

    for k in range(6):
        accuracy = np.zeros((len(contid_set),len(np.logspace(0,np.log10(128)))))
        for j in range(len(contid_set)): 
            contid = contid_set[j]
            y_fit = np.load('./dvsc/stimulus_decoding/svm_results/fitted_lines_stim_categories/'+str(contid)+'_'+str(k)+'.npy')
            accuracy[j] = y_fit
        average = np.nanmean(accuracy * 100, axis = 0)
        error = np.nanstd(accuracy * 100, axis = 0)/(np.sqrt(len(contid_set)))
        col = stim_colors[k]
        acc.plot(np.log10(np.logspace(0,np.log10(128))), average, c = col, lw = 1)
        acc.fill_between(np.log10(np.logspace(0,np.log10(128))), average-error, average+error, facecolor = col, alpha = 0.4)
    acc.set_title(depth_names[w] + ' (n = ' + str(len(imaging_depths[w])) + ')',fontsize=fs-2.5)
    plt.xticks(np.log10(neuron_levels),labels,fontsize=fs-2)
    plt.yticks(fontsize=fs-2)
plt.tight_layout()
plt.savefig('./dvsc/figures/stimulus_decoding_imaging_depth_stimuli.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

##### Comparing SVM and MLR classification accuracy

In [ ]:
fig = plt.figure(figsize=(3,3),dpi=400)
for contid in all_ecs_ids:
    svm = 100 * np.load('./dvsc/stimulus_decoding/svm_results/interpolated_scores/'+str(contid)+'.npy')
    mlr = 100 * np.load('./dvsc/stimulus_decoding/mlr_results/interpolated_scores/'+str(contid)+'.npy')    
    plt.plot(mlr,svm,'o', markersize = 2, alpha = 0.7)
plt.plot(np.linspace(0,100), np.linspace(0,100), color = 'k', lw = 1)
fs = 10
plt.xlabel('Multinomial Logistic Regression', fontsize = fs)
plt.ylabel('Linear Support Vector Machine', fontsize = fs)
plt.xticks(np.arange(0,101,25), fontsize = fs)
plt.yticks(np.arange(0,101,25), fontsize = fs)
plt.title('Accuracy Comparison for \n Stimulus Decoding', fontsize = fs)
plt.savefig('./dvsc/figures/accuracy_comparison_svm_mlr.pdf',format = 'pdf', bbox_inches = 'tight')
plt.show()

Orientation and Direction Selectivity Index Results
---

In [ ]:
for contid in all_ecs_ids:
    exps = boc.get_ophys_experiments(experiment_container_ids=[contid])
    if os.path.exists('./dvsc/selectivity/osi/' + str(contid) + '.npy') and os.path.exists('./dvsc/selectivity/dsi/' + str(contid) + '.npy'):
        continue
    for i in range(3):
        if exps[i]['session_type'] == 'three_session_A':
            idA = exps[i]['id']
    dsA = boc.get_ophys_experiment_data(idA)
    cell_data = dg.DriftingGratings(dsA)
    data_table = cell_data.get_peak()
    osi = data_table.osi_dg.tolist()
    np.save('./dvsc/selectivity/osi/' + str(contid), osi)
    
    dsi = data_table.dsi_dg.tolist()
    np.save('./dvsc/selectivity/dsi/' + str(contid), dsi)

### OSI and DSI by visual area

In [ ]:
if os.path.exists('./dvsc/selectivity/means/area_osi_area.npy'):
    dsi_means = np.load('./dvsc/selectivity/means/area_dsi.npy')
    dsi_error = np.load('./dvsc/selectivity/means/area_dsi_error.npy')
    osi_means = np.load('./dvsc/selectivity/means/area_osi.npy')
    osi_error = np.load('./dvsc/selectivity/means/area_osi_error', osi_error)
else:
    dsi_means = []
    dsi_error = []
    osi_means = []
    osi_error = []
    for k in range(len(cortical_areas)):
        id_list = cortical_areas[k]
        area_dsi = []
        area_osi = []
        for j in range(len(id_list)):
            area_dsi.append(np.load('./dvsc/selectivity/dsi/'+str(id_list[j])+'.npy'))
            area_osi.append(np.load('./dvsc/selectivity/osi/'+str(id_list[j])+'.npy'))

        dsi_means.append(np.mean(np.concatenate(area_dsi)))
        dsi_error.append(np.std(np.concatenate(area_dsi))/np.sqrt(len(np.concatenate(area_dsi))))
        osi_means.append(np.mean(np.concatenate(area_osi)))
        osi_error.append(np.std(np.concatenate(area_osi))/np.sqrt(len(np.concatenate(area_osi))))

    np.save('./dvsc/selectivity/means/area_dsi', dsi_means)
    np.save('./dvsc/selectivity/means/area_dsi_error', dsi_error)
    np.save('./dvsc/selectivity/means/area_osi', osi_means)
    np.save('./dvsc/selectivity/means/area_osi_error', osi_error)

fig = plt.figure (figsize=(4,3),dpi=400)
ind = np.arange(0,6)
plt.ylim(-0.1,2.1)
plt.ylabel("Mean Orientation Selectivity Index", fontsize = 9)
plt.title("Orientation Selectivity")
plt.xticks(ind, area_names)
for i in range(6):
    plt.bar(ind[i],osi_means[i],width=0.5,yerr=osi_error[i],color=area_colors[i])
plt.savefig('./dvsc/figures/osi_area.pdf', format = 'pdf', bbox_inches = 'tight')
plt.show()

fig = plt.figure (figsize=(4,3),dpi=400)
plt.ylim(-0.1,2.1)
plt.ylabel("Mean Direction Selectivity Index", fontsize = 9)
plt.title("Direction Selectivity")
plt.xticks(ind, area_names)
for i in range(6):
    plt.bar(ind[i],dsi_means[i],width=0.5,yerr=dsi_error[i],color=area_colors[i])
    
plt.savefig('./dvsc/figures/dsi_area.pdf', format = 'pdf', bbox_inches = 'tight')
plt.show()

### OSI and DSI by depth 

In [ ]:
if os.path.exists('./dvsc/selectivity/means/depth_osi_area.npy'):
    dsi_means = np.load('./dvsc/selectivity/means/depth_dsi.npy')
    dsi_error = np.load('./dvsc/selectivity/means/depth_dsi_error.npy')
    osi_means = np.load('./dvsc/selectivity/means/depth_osi.npy')
    osi_error = np.load('./dvsc/selectivity/means/depth_osi_error', osi_error)
else:
    dsi_means = []
    dsi_error = []
    osi_means = []
    osi_error = []
    for k in range(len(cortical_areas)):
        id_list = cortical_areas[k]
        area_dsi = []
        area_osi = []
        for j in range(len(id_list)):
            area_dsi.append(np.load('./dvsc/selectivity/dsi/'+str(id_list[j])+'.npy'))
            area_osi.append(np.load('./dvsc/selectivity/osi/'+str(id_list[j])+'.npy'))

        dsi_means.append(np.mean(np.concatenate(area_dsi)))
        dsi_error.append(np.std(np.concatenate(area_dsi))/np.sqrt(len(np.concatenate(area_dsi))))
        osi_means.append(np.mean(np.concatenate(area_osi)))
        osi_error.append(np.std(np.concatenate(area_osi))/np.sqrt(len(np.concatenate(area_osi))))

    np.save('./dvsc/selectivity/means/depth_dsi', dsi_means)
    np.save('./dvsc/selectivity/means/depth_dsi_error', dsi_error)
    np.save('./dvsc/selectivity/means/depth_osi', osi_means)
    np.save('./dvsc/selectivity/means/depth_osi_error', osi_error)

fig = plt.figure (figsize=(4,3),dpi=400)
ind = np.arange(0,6)
plt.ylim(-0.1,2.1)
plt.ylabel("Mean Orientation Selectivity Index", fontsize = 9)
plt.title("Orientation Selectivity")
plt.xticks(ind, depth_names, rotation = 20)
for i in range(4):
    plt.bar(ind[i],osi_means[i],width=0.5,yerr=osi_error[i],color=depth_colors[i])
plt.savefig('./dvsc/figures/osi_depth.pdf', format = 'pdf', bbox_inches = 'tight')
plt.show()

fig = plt.figure (figsize=(4,3),dpi=400)
plt.ylim(-0.1,2.1)
plt.ylabel("Mean Direction Selectivity Index", fontsize = 9)
plt.title("Direction Selectivity")
plt.xticks(ind, depth_names, rotation = 20)
for i in range(4):
    plt.bar(ind[i],dsi_means[i],width=0.5,yerr=dsi_error[i],color=depth_colors[i])
    
plt.savefig('./dvsc/figures/dsi_depth.pdf', format = 'pdf', bbox_inches = 'tight')
plt.show()